In [2]:
# Installing Interactive Brokers API

# This is not installed with pip, instead you must install TWS or IB Gateway 
# then install the TWS API and install via the following commands
# `python -m pip install setuptools`
# `cd C:\TWS API\source\pythonclient\`
# `python setup.py install`
import ibapi

In [3]:
import logging
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import threading

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.historical_data_end = threading.Event()  # Completion flag

    def historicalData(self, reqId, bar):
        logging.info(f"Received data for {bar.date}")
        self.data.append([bar.date, bar.close, bar.open, bar.volume, bar.high, bar.low])

    def historicalDataEnd(self, reqId, start, end):
        logging.info(f"Historical data request ended for reqId: {reqId}")
        self.historical_data_end.set()  # Set the flag when data ends

def run_loop():
    app.run()

# Establish connection to IB
app = IBApi()
app.connect("127.0.0.1", 7497, clientId=123)
logging.info("Connecting to Interactive Brokers API...")
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# Wait for the connection to be established
while not app.isConnected():
    logging.info("Waiting for connection...")
    threading.Event().wait(1)  # Wait for 1 second before checking again

# Define the contract
contract = Contract()
contract.symbol = "SPY"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"
logging.info("Contract defined for SPY.")

# Request historical data
app.reqHistoricalData(reqId=1, contract=contract,
                      endDateTime='', durationStr='1 Y',
                      barSizeSetting='1 day', whatToShow='TRADES',
                      useRTH=True, formatDate=1, keepUpToDate=False,
                      chartOptions=[])
logging.info("Requested historical data for SPY.")

# Wait for historical data to be fetched
if not app.historical_data_end.wait(timeout=60):  # Timeout after 60 seconds
    logging.warning("Timed out waiting for historical data.")
else:
    logging.info("Historical data request completed.")

# Disconnect
app.disconnect()
logging.info("Disconnected from Interactive Brokers API.")

# Create DataFrame
df = pd.DataFrame(app.data, columns=['Date', 'Close', 'Open', 'Volume', 'High', 'Low'])
print(df)


2023-12-27 13:56:03,079 - INFO - sent startApi
2023-12-27 13:56:03,081 - INFO - REQUEST startApi {}
2023-12-27 13:56:03,081 - INFO - SENDING startApi b'\x00\x00\x00\n71\x002\x00123\x00\x00'
2023-12-27 13:56:03,082 - INFO - ANSWER connectAck {}
2023-12-27 13:56:03,083 - INFO - Connecting to Interactive Brokers API...
2023-12-27 13:56:03,085 - INFO - ANSWER managedAccounts {'accountsList': 'DU7625017'}
2023-12-27 13:56:03,087 - INFO - Contract defined for SPY.
2023-12-27 13:56:03,089 - INFO - ANSWER nextValidId {'orderId': 1}
2023-12-27 13:56:03,091 - INFO - REQUEST reqHistoricalData {'reqId': 1, 'contract': 1910619014496: 0,SPY,STK,,0,,,SMART,,USD,,,False,,,,combo:, 'endDateTime': '', 'durationStr': '1 Y', 'barSizeSetting': '1 day', 'whatToShow': 'TRADES', 'useRTH': True, 'formatDate': 1, 'keepUpToDate': False, 'chartOptions': []}
2023-12-27 13:56:03,091 - INFO - ANSWER error {'reqId': -1, 'errorCode': 2104, 'errorString': 'Market data farm connection is OK:usfarm.nj', 'advancedOrderRej

         Date   Close    Open    Volume    High     Low
0    20221228  376.66  381.34  45531220  383.39  376.42
1    20221229  383.44  379.66  43269617  384.35  379.08
2    20221230  382.43  380.64  50712172  382.53  378.43
3    20230103  380.82  384.42  49025780  386.43  377.83
4    20230104  383.76  383.18  59492813  385.88  380.00
..        ...     ...     ...       ...     ...     ...
246  20231220  468.26  473.96  67376027  475.90  467.82
247  20231221  472.70  471.32  57760976  472.98  468.84
248  20231222  473.65  473.86  44344028  475.38  471.70
249  20231226  475.65  474.08  31409570  476.58  473.99
250  20231227  476.35  475.44  19580113  476.66  475.08

[251 rows x 6 columns]
